In [8]:
import numpy as np
import pandas as pd

###  FoodMatrix contains metabalomic profile of each food. 
### FoodList corresponds to  foods of each column in food matrix
### FoodComposition contains the ingredients of each complex food along with all ingredients

In [9]:
FoodMatrix = np.load("FoodCompositionsMatrix.npy")

In [10]:
FoodList = np.load("UniqueFilesNamesForFoods.npy", allow_pickle = True)

In [11]:
FoodComposition = pd.read_csv("FoodCompositions.csv")

In [12]:
FoodCompositionsArray = FoodComposition.values

### create seperate array for complex foods and ingredients

In [13]:
from sklearn.linear_model import LogisticRegression

In [ ]:
BasicArray = FoodCompositionsArray[1:2,:] * 0
CompositionArray = FoodCompositionsArray[1:2,:] * 0

BasicFoodMatrix = np.zeros((95006,1))
ComposedFoodMatrix = np.zeros((95006,1))


for k in range(0,3534):
    temp = FoodCompositionsArray[k,3]
    if(temp != temp):
        temp = "hi"
    split_comma= temp.split(',')
    split_comma_space = split_comma[0].split(' ')
    OtherIndicator  = FoodCompositionsArray[k,4]


    FoodType = 0 

    if(OtherIndicator != OtherIndicator or OtherIndicator == "not applicable" or OtherIndicator == 'not provided' or OtherIndicator == 'not collected'):
        if(len(split_comma) <=1 and len(split_comma_space) <=2):
            FoodType = "basic"
        if(len(split_comma_space) <=2 and len(split_comma) <=1):
            FoodType = "basic"
            
    if(FoodType == "basic"):
        temp = FoodCompositionsArray[k:k+1,:]
        BasicArray = np.vstack((BasicArray,temp))
        val = temp[0,0]
        loc = np.where(FoodList == val)[0][0]
        temp = FoodMatrix[:,loc:loc+1]
        BasicFoodMatrix = np.hstack((BasicFoodMatrix,temp))
    else:
        temp = FoodCompositionsArray[k:k+1,:]
        CompositionArray = np.vstack((CompositionArray,temp))
        val = temp[0,0]
        loc = np.where(FoodList == val)[0][0]
        temp = FoodMatrix[:,loc:loc+1]
        ComposedFoodMatrix = np.hstack((ComposedFoodMatrix,temp))
        
    print(k)



BasicArray = np.delete(BasicArray,0,axis =0)
CompositionArray = np.delete(CompositionArray,0,axis =0)

BasicFoodMatrix = np.delete(BasicFoodMatrix,0,axis =1)
ComposedFoodMatrix = np.delete(ComposedFoodMatrix,0,axis =1)


### employ optimization procedure to find ingredients of each complex food

In [77]:
from sklearn.linear_model import LogisticRegression

In [102]:
A = np.zeros((1682,12)).astype('O')

In [ ]:
for itera in range(0,1682):
    kstart = itera
    cstar = .0008
    num = 10
    if(np.sum(ComposedFoodMatrix[:,kstart]) == 0):
        continue
    while num >5:
        #kstart = np.where(CompositionArray[:,0] == D)[0][0]
        cstar = cstar - .00003 
        clf = LogisticRegression(random_state=0, penalty = 'l1', C = cstar, class_weight = "balanced").fit(BasicFoodMatrix,ComposedFoodMatrix[:,kstart])
        temp = np.where(clf.coef_!=0)[1]
        num = temp.shape[0]
    
    print(cstar,itera)
    clf = LogisticRegression(random_state=0, penalty = 'l1', C = cstar, class_weight = "balanced").fit(BasicFoodMatrix,ComposedFoodMatrix[:,kstart])
    temp = np.where(clf.coef_!=0)[1]
        
    t = BasicArray[temp,2]
    A[itera,0] = CompositionArray[itera,0]
    A[itera,1] = CompositionArray[itera,2]
    A[itera,2] = CompositionArray[itera,3]
    A[itera,3] = CompositionArray[itera,4]
    A[itera,4] = CompositionArray[itera,5]
    A[itera,5] = CompositionArray[itera,6]
    A[itera,6] = CompositionArray[itera,7]
    for y in range(0,len(t)):
        A[itera,y+7] = t[y]
    


In [123]:
np.save("ResultsComplexFoods", A)

In [154]:
np.save("IngredientNames",BasicArray[:,2])

In [155]:
t = np.load("IngredientNames.npy", allow_pickle = True )